In [3]:
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
import numpy as np
import pandas as pd
import glob

In [4]:
num_exp = 1
dataset = "king"
real_path = "Real_Datasets/king.csv"
fake_file_root = "Fake_Datasets"

In [5]:
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = ['bedrooms', "floors", 'waterfront', 'view', 'condition', 'grade','zipcode'],   
                 log_columns = [],
                 mixed_columns= {"sqft_basement":[0.0],  "yr_renovated":[0.0]},
                 general_columns= ["bathrooms", "sqft_living", "sqft_above", "yr_built", "long", "sqft_living15"],
                 non_categorical_columns= [],
                 integer_columns = [],
                 problem_type= {"Regression": "price"}) 

for i in range(num_exp):
    synthesizer.fit()

  0%|                                                                                                                                     | 0/150 [00:00<?, ?it/s]

generator structure:  Generator(
  (seq): Sequential(
    (0): ConvTranspose2d(266, 256, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (1): LayerNorm((256, 2, 2), eps=1e-05, elementwise_affine=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): LayerNorm((128, 4, 4), eps=1e-05, elementwise_affine=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): LayerNorm((64, 8, 8), eps=1e-05, elementwise_affine=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  )
)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [12:06<00:00,  4.84s/it]

Finished training in 738.5056114196777  seconds.


In [6]:
for i in range(3):
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}_ctabplus.csv".format(exp=i), index= False)

generated result length:  (22396, 20)
generated result length:  (21941, 20)
generated result length:  (22424, 20)


In [8]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [9]:
model_dict =  {"Regression":["l_reg","ridge","lasso","B_ridge"]}
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",model_dict, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Mean_Absolute_Percentage_Error","Explained_Varaince_Score","R2_Score"])
result_df.index = list(model_dict.values())[0]
result_df

,Mean_Absolute_Percentage_Error,Explained_Varaince_Score,R2_Score
l_reg,0.015166,0.020321,0.037300
ridge,0.015466,0.021220,0.037621
lasso,0.015587,0.020319,0.037342
B_ridge,0.015699,0.020517,0.037326


In [10]:
# ctabplus
insurance_categorical = ['bedrooms', "floors", 'waterfront', 'view', 'condition', 'grade','zipcode']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,insurance_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

column:  date WD:  0.021664532323824
column:  bedrooms JSD:  0.07406527406706762
column:  bathrooms WD:  0.016980764394344122
column:  sqft_living WD:  0.014219149021258948
column:  sqft_lot WD:  0.0011990821509605576
column:  floors JSD:  0.034618949536077644
column:  waterfront JSD:  0.0270201447597129
column:  view JSD:  0.037729996377866726
column:  condition JSD:  0.033514058559008686
column:  grade JSD:  0.08800911188988236
column:  sqft_above WD:  0.011549294545643959
column:  sqft_basement WD:  0.0034711576866429446
column:  yr_built WD:  0.007681289252351483
column:  yr_renovated WD:  0.014927505480426317
column:  zipcode JSD:  0.16489880248868577
column:  lat WD:  0.02657911382138573
column:  long WD:  0.014846288664317359
column:  sqft_living15 WD:  0.02384116692949852
column:  sqft_lot15 WD:  0.00115209156828118
column:  price WD:  0.0008783043460760184
column:  date WD:  0.02141006038706135
column:  bedrooms JSD:  0.07569138670242004
column:  bathrooms WD:  0.0167855969289

,Average WD (Continuous Columns,Average JSD (Categorical Columns),Correlation Distance
0,0.012414,0.066046,0.930768
